In [1]:
import tvm
import tvm.relay as relay
from tvm.contrib.download import download_testdata
from tvm.contrib import graph_executor
from tvm.contrib.debugger import debug_executor
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import preprocess_input
import time
import numpy as np
from PIL import Image
import sys
import os

import json
import pygraphviz as pgv

from simple_unet import UNet

def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]), node['name'] + '[{}]'.format(node_idx))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')
    # return jupyterImage(A.draw(format='png', prog='dot'))

img_url = "https://github.com/dmlc/mxnet.js/blob/main/data/cat.png?raw=true"
img_path = download_testdata(img_url, "cat.png", module="data")
img = Image.open(img_path).resize((512, 512))

model = UNet(in_dim=3, out_dim=1, num_filter=16)

# Preprocess (data type conversion to float32, NCHW order)
img_data = np.array(img)[np.newaxis, :].astype("float32")
img_data = preprocess_input(img_data).transpose([0, 3, 1, 2])

# Get IRModule and Parameters
shape_dict = {"input_1": img_data.shape}
mod, params = relay.frontend.from_keras(model, shape_dict)

target = 'cuda'

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

local_dev = tvm.cuda()

show_graph(lib['get_graph_json'](), file_name='unet')

ImportError: cannot import name 'graph_executor' from 'tvm.contrib' (/home/j/tvm-0.7.0/python/tvm/contrib/__init__.py)

In [2]:
tvm.__version__

'0.7.0'

In [3]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 512, 512, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 256, 256, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 32)      0         
Total params: 5,088
Trainable params: 5,088
Non-trainable params: 0
_________________________________________________________________


In [1]:
!nvidia-smi

Fri Dec 24 14:57:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 45%   27C    P8    N/A /  75W |    105MiB /  4038MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------